#### This notebook continues my previous work.
#### If you trained model with https://www.kaggle.com/code/alexeyolkhovikov/segformer-training, you can use this notebook for inference.
#### If you find this notebook useful, please upvote!

In [1]:
!cp -r ../input/pytorch-segmentation-models-lib/ ./
!cp -r ../input/torchmetrics/ ./

!pip config set global.disable-pip-version-check true

!pip install -q ./pytorch-segmentation-models-lib/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4
!pip install -q ./pytorch-segmentation-models-lib/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3
!pip install -q ./pytorch-segmentation-models-lib/timm-0.4.12-py3-none-any.whl
!pip install -q ./pytorch-segmentation-models-lib/segmentation_models_pytorch-0.2.0-py3-none-any.whl
!pip install -q ./torchmetrics/torchmetrics-0.9.1-py3-none-any.whl

Writing to /root/.config/pip/pip.conf


In [2]:
import pandas as pd
import numpy as np

import os
import json
from tqdm.auto import tqdm
import gc

from skimage import io
from skimage.transform import resize
from PIL import Image

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

import pytorch_lightning as pl
from pytorch_lightning import Trainer
import segmentation_models_pytorch as smp
from torchmetrics import Dice

In [3]:
TEST_IMG_PATH = "../input/hubmap-organ-segmentation/test_images/"

MODEL_PATH = [
    "../input/unosegformer/Segformer",
]

In [4]:
def mask_to_rle(mask):
    #Rescale image to original size
    size = int(len(mask.flatten())**.5)
    n = Image.fromarray(mask.reshape((size, size))*255.0)
    n = np.array(n).astype(np.float32)
    #Get pixels to flatten
    pixels = n.T.flatten()
    #Round the pixels using the half of the range of pixel value
    pixels = (pixels-min(pixels) > ((max(pixels)-min(pixels))/2)).astype(int)
    pixels = np.nan_to_num(pixels) #incase of zero-div-error
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [5]:
class SegmentationModel(pl.LightningModule):
    def __init__(
        self,
        model
        ):
        super(SegmentationModel, self).__init__()
        
        self.model = model
        
        self.criterion = smp.losses.DiceLoss(
            smp.losses.BINARY_MODE,
            from_logits=True,
            log_loss=True
        )

        self.metric = Dice(
            num_classes=1,
            average='none'
        )
        
    def forward(self, image, size):
        outputs = self.model(image)
        
        upsampled_logits = nn.functional.interpolate(
            outputs.logits,
            size=size, 
            mode="bilinear",
            align_corners=False
        )
        
        return upsampled_logits
        

    def training_step(self, batch, batch_idx):
        image, mask = batch[0], batch[1]
        outputs = self.model(pixel_values=image, labels=mask.long())
        
        upsampled_logits = nn.functional.interpolate(
            outputs.logits,
            size=mask.shape[-2:], 
            mode="bilinear",
            align_corners=False
        )
        
        loss = outputs.loss
        
        return {'loss': loss, 'logits_mask': upsampled_logits, 'mask': mask}
    
    def training_epoch_end(self, outputs):
        loss = [item['loss'].item() for item in outputs]
        logits_mask = torch.cat([item['logits_mask'] for item in outputs]).sigmoid()
        mask = torch.cat([item['mask'] for item in outputs])
        
        pred_mask = logits_mask.argmax(dim=1).float()
        
        tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), mask.long(), mode="binary")
        per_image_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        
        log_parameters = {
            "loss_train": np.mean(loss),
            "per_image_iou_train": per_image_iou,
            "dataset_iou_train": dataset_iou,
        }
        
        self.log_dict(log_parameters)
    
    def validation_step(self, batch, batch_idx):
        image, mask = batch[0], batch[1]
        outputs = self.model(pixel_values=image, labels=mask.long())
        
        upsampled_logits = nn.functional.interpolate(
            outputs.logits,
            size=mask.shape[-2:], 
            mode="bilinear",
            align_corners=False
        )
        
        loss = outputs.loss
        
        return {'loss': loss, 'logits_mask': upsampled_logits, 'mask': mask}
        
    def validation_epoch_end(self, outputs):
        loss = torch.from_numpy(np.array([item['loss'].item() for item in outputs]))
        logits_mask = torch.cat([item['logits_mask'] for item in outputs]).sigmoid()
        mask = torch.cat([item['mask'] for item in outputs])
        
        pred_mask = logits_mask.argmax(dim=1).float()
        
        tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), mask.long(), mode="binary")
        per_image_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro-imagewise")
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        
        log_parameters = {
            "loss_valid": torch.mean(loss),
            "per_image_iou_valid": per_image_iou,
            "dataset_iou_valid": dataset_iou,
        }
        
        self.log_dict(log_parameters)        

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = PeakScheduler(
            optimizer,
            lr_ramp_ep=int(Config.EPOCHS * 0.25), 
            lr_decay=0.95,
            lr_max=1e-03,
            lr_min=1e-06
        )
        
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "interval": "epoch", "frequency": 1}
        }

In [6]:
models = []

for p in MODEL_PATH:
    net = SegformerForSemanticSegmentation.from_pretrained(p)
    model = SegmentationModel(net)
    model = model.to("cuda")
    models.append(model)
    

In [7]:
IMG_SIZE = 768

class CustomDatasetTest(Dataset):
    def __init__(
        self, 
        paths: str = None,
        img_size: int = None
        ):
        
        self.paths = paths
        
        self.transform = A.Compose([
            A.Resize(img_size, img_size),
            A.Normalize()
        ])

        self.to_tensor = ToTensorV2()
        self.img_size = img_size
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(
        self, 
        idx: int = None
    ):
        img = np.asarray(Image.open(self.paths[idx]))
        
        H, W = img.shape[:2]
        
        transformed = self.transform(image=img)
        
        return {
            'image': self.to_tensor(image=transformed['image'].copy())['image'],
#             'image_small': self.to_tensor(image=A.Normalize()(image=A.Resize(512, 512)(image=img)['image'])['image'])['image'],
#             'image_big': self.to_tensor(image=A.Normalize()(image=A.Resize(1024, 1024)(image=img)['image'])['image'])['image'],
#             'image_blurred': self.to_tensor(image=A.Blur(p=1.)(image=transformed['image'].copy())['image'])['image'],
#             'image_jittered': self.to_tensor(image=A.ColorJitter(p=1.)(image=transformed['image'].copy())['image'])['image'],
            'image_num': self.paths[idx].split("/")[-1].split(".")[0],
            'initial_size': (H, W)
        }

In [8]:
paths = [os.path.join(TEST_IMG_PATH, img) for img in os.listdir(TEST_IMG_PATH)]
dataset_test = CustomDatasetTest(paths, IMG_SIZE)
dataloader_test = DataLoader(dataset_test, batch_size=1, drop_last=False)

In [9]:
def rotate(img, k, model):
    logits_rot = model.forward(
        torch.rot90(img, k=k, dims=[2, 3]).to("cuda", dtype=torch.float32),
        size=batch['initial_size']
    ).to("cpu") 
            
    logits_rot = torch.rot90(logits_rot, k=k, dims=[3, 2])
    
    return logits_rot  

def flip_vertical(img, model):
    logits_flipped = model.forward(
        torch.flip(img, dims=[3]).to("cuda", dtype=torch.float32),
        size=batch['initial_size']
    ).to("cpu") 
    
    logits_flipped = torch.flip(logits_flipped, dims=[3])

    return logits_flipped

def flip_horizontal(img, model):
    logits_flipped = model.forward(
        torch.flip(img, dims=[1, 2]).to("cuda", dtype=torch.float32),
        size=batch['initial_size']
    ).to("cpu") 
    
    logits_flipped = torch.flip(logits_flipped, dims=[1, 2])

    return logits_flipped

In [10]:
ids = []
rle = []

for _, batch in enumerate(tqdm(dataloader_test)):
    preds = []
    with torch.no_grad():
        for model in models:
            model.eval()
            
            img = batch['image']
            
            logits_image = model.forward(
                img.to("cuda", dtype=torch.float32),
                size=batch['initial_size']
            ).to("cpu")
            
#             logits_rot_90 = rotate(img, 1, model)
#             logits_rot_180 = rotate(img, 2, model)
#             logits_rots_270 = rotate(img, 3, model)
            
            logits_flipped_H = flip_horizontal(img, model)
            logits_flipped_V = flip_vertical(img, model)

        
            preds.extend([logits_image, logits_flipped_H, logits_flipped_V]) #, logits_rot_90, logits_rot_180, logits_rots_270
        
    preds = torch.mean(torch.cat(preds, dim=0), dim=0).unsqueeze(0)
    
    
    
    pr_masks = preds.sigmoid().argmax(1)
    
    
        
    ids.append(batch['image_num'][0])
    rle.append(mask_to_rle(pr_masks.numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
submission = {
    "id": ids,
    "rle": rle
}

In [12]:
submission = pd.DataFrame.from_dict(submission)
submission.to_csv("submission.csv", index=False)

In [13]:
submission

,id,rle
0,10078,390 119 2413 119 4436 119 6459 119 8482 119 10...
